
## Introducción a los componentes fundamentales de Langmem: `MemoryStoreManager` y `MemoryManager`

En el ecosistema de **Langmem**, la gestión de la memoria es un aspecto crucial para el funcionamiento eficiente y coherente de agentes inteligentes y sistemas conversacionales. Dos bloques fundamentales en esta arquitectura son:

- **MemoryStoreManager**: Responsable de gestionar el almacenamiento físico de los recuerdos o memorias. Administra las distintas fuentes de almacenamiento, ya sea en memoria local, bases de datos, o sistemas de almacenamiento distribuidos.

- **MemoryManager**: Actúa como el orquestador lógico de las memorias. Controla cómo se crean, actualizan, recuperan y eliminan las memorias en función del contexto y las necesidades del agente.

En esta sección, exploraremos en detalle el rol y la interacción de estos dos componentes, entendiendo cómo permiten a Langmem construir agentes más adaptativos, contextuales y persistentes en su conocimiento.


In [4]:
# Importamos las clases y tipos necesarios de langgraph
from langgraph.store.base import BaseStore, Item, Op, Result
from langgraph.store.memory import InMemoryStore
from typing import Any, Iterable, Literal, NamedTuple, Optional, Union, cast
from langchain_aws import ChatBedrockConverse
import boto3
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables.config import RunnableConfig
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model
from langgraph.func import entrypoint
from langgraph.store.memory import InMemoryStore
from langmem import create_memory_store_manager
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool
from langchain_aws import ChatBedrockConverse
from langchain_aws import ChatBedrock
from IPython.display import Image, display
from langmem import create_memory_manager # 
from pydantic import BaseModel
# Definimos una clase personalizada que extiende de BaseStore
class CustomMemoryStore(BaseStore):
    """
    CustomMemoryStore es una implementación personalizada de BaseStore que
    actúa como adaptador para un almacenamiento externo (ext_store).
    Permite operaciones de get, put y batch de forma síncrona y asíncrona.
    """

    def __init__(self, ext_store):
        """
        Inicializa el CustomMemoryStore con un almacén externo.

        Args:
            ext_store: Un objeto que implementa los métodos get, put y batch.
        """
        self.store = ext_store

    def get(self, namespace: tuple[str, ...], key: str) -> Optional[Item]:
        """
        Recupera un ítem del almacén dado un namespace y una clave.

        Args:
            namespace (tuple): Un espacio de nombres como tupla de strings.
            key (str): La clave del ítem a recuperar.

        Returns:
            Optional[Item]: El ítem encontrado o None si no existe.
        """
        return self.store.get(namespace, key)

    def put(self, namespace: tuple[str, ...], key: str, value: dict[str, Any]) -> None:
        """
        Guarda un ítem en el almacén bajo un namespace y una clave.

        Args:
            namespace (tuple): El espacio de nombres donde guardar.
            key (str): La clave bajo la cual guardar el ítem.
            value (dict): El valor del ítem a guardar.
        """
        print(f"PUT::namespace={namespace}, key={key}, value={value}:")
        return self.store.put(namespace, key, value)

    async def aget(self, namespace: tuple[str, ...], key: str) -> Optional[Item]:
        """
        Recupera un ítem de manera asíncrona.

        Nota: Aunque se define como async, internamente llama al método síncrono get.

        Args:
            namespace (tuple): Espacio de nombres.
            key (str): Clave a recuperar.

        Returns:
            Optional[Item]: El ítem encontrado o None.
        """
        res = await self.get(namespace, key)  # Puede lanzar advertencias ya que get no es async
        return res

    async def aput(self, namespace: tuple[str, ...], key: str, value: dict[str, Any]) -> None:
        """
        Guarda un ítem de forma asíncrona.

        Nota: Llama al método síncrono put ya que no existe una versión nativa async.

        Args:
            namespace (tuple): Espacio de nombres.
            key (str): Clave donde guardar.
            value (dict): El valor a guardar.
        """
        res = self.put(namespace, key, value)  # No se puede await porque put es síncrono
        print(f"ASYN-PUT::::namespace={namespace}, key={key}, value={value}:")
        return None

    def batch(self, ops: Iterable[Op]) -> list[Result]:
        """
        Ejecuta una serie de operaciones en lote de forma síncrona.

        Args:
            ops (Iterable[Op]): Lista de operaciones a realizar.

        Returns:
            list[Result]: Resultados de las operaciones.
        """
        return self.store.batch(ops)

    async def abatch(self, ops: Iterable[Op]) -> list[Result]:
        """
        Ejecuta una serie de operaciones en lote de manera asíncrona.

        Args:
            ops (Iterable[Op]): Lista de operaciones a realizar.

        Returns:
            list[Result]: Resultados de las operaciones.
        """
        print(f"ASYNC::BATCH::::ops={ops}:")
        res = await self.store.abatch(ops)
        return res


- Buena práctica: Podrías considerar envolver self.get en aget usando asyncio.to_thread para evitar advertencias de tipo (aunque depende del rendimiento que quieras optimizar).

In [6]:
class Triple(BaseModel):
    """
    Representa una estructura de triple (sujeto, predicado, objeto) utilizada para almacenar hechos, 
    preferencias y relaciones como triples en una base de datos o sistema de almacenamiento.
    Un triple tiene un contexto opcional que ayuda a clasificar la información.

    Atributos:
    - subject: str
      El sujeto del triple (por ejemplo, una entidad o persona).
    - predicate: str
      El predicado que describe la relación entre el sujeto y el objeto (por ejemplo, "gestiona", "es miembro de").
    - object: str
      El objeto que está relacionado con el sujeto (por ejemplo, otra persona, cosa o concepto).
    - context: str | None
      Un contexto opcional que puede usarse para proporcionar más detalles sobre el triple. Por defecto es None.
    """
    subject: str  # Sujeto del triple
    predicate: str  # Relación o acción
    object: str  # Objeto del triple
    context: str | None = None  # Contexto opcional, por defecto es None


# Establecer el almacenamiento en memoria
in_memory_store = CustomMemoryStore(InMemoryStore())
# ---- ⚠️ Actualiza la región para tu configuración de AWS ⚠️
region = 'us-east-2'  # Región de AWS para el cliente de Bedrock

# Crear un cliente de AWS para el servicio de Bedrock
bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,  # Usamos la región de AWS previamente configurada
)

# Crear el cliente específico para la API de Bedrock
bedrock_client = boto3.client("bedrock-runtime", region_name=region)

# Definir el modelo que se va a usar (en este caso, un modelo de lenguaje de Anthropic)
model_id = "us.anthropic.claude-3-5-haiku-20241022-v1:0"

# Crear un objeto ChatBedrockConverse que se usará para invocar el modelo de lenguaje
llm = ChatBedrockConverse(
    model=model_id,  # Identificador del modelo
    temperature=0,  # Controla la creatividad del modelo (0 significa determinístico)
    max_tokens=5000,  # Número máximo de tokens a generar
    client=bedrock_client,  # Cliente de AWS para hacer las llamadas
)

# Configurar un MemoryManager para gestionar el almacenamiento de triples (hechos y relaciones)
memory_manager = create_memory_store_manager(
    llm,  # El modelo de lenguaje que usará el manager
    namespace=("chat", "{user_id}", "triples"),  # Espacio de nombres único para cada usuario
    schemas=[Triple],  # Usamos el esquema de Triple definido anteriormente
    instructions="Extract all user information and events as triples.",  # Instrucciones para extraer la información como triples
    enable_inserts=True,  # Permitir insertar nuevos triples en la memoria
    enable_deletes=True,  # Permitir eliminar triples de la memoria
)

# Ejemplo de un mensaje de usuario para ser procesado por el modelo
messages = [
    {
        "role": "user",  # Especificamos que es un mensaje del usuario
        "content": "Alice manages the ML team and mentors Bob, who is also on the team.",  # Contenido del mensaje
    },
]

# Se comenta temporalmente esta parte porque se necesita un contexto adecuado para que funcione correctamente.
# El código fallará si se invoca fuera de un contexto ejecutable de tipo "RunnableContext".
# memory_manager.invoke({"messages": messages})

# Definir la aplicación con el contexto de almacenamiento en memoria
@entrypoint(store=in_memory_store)  # El decorador `entrypoint` permite que la función `app` reciba el almacenamiento
def app(messages: list):
    """
    Función principal de la aplicación que invoca el modelo de lenguaje y procesa los mensajes del usuario.

    Parámetros:
    - messages: list
      Lista de mensajes que se pasan al modelo para obtener una respuesta.

    Retorna:
    - La respuesta del modelo de lenguaje basado en los mensajes proporcionados.
    """
    # Invocar el modelo de lenguaje (ChatGPT o similar) con los mensajes
    response = llm.invoke(
        [
            {
                "role": "system",  # Mensaje del sistema que da instrucciones al modelo
                "content": "You are a helpful assistant.",  # El rol del modelo como asistente
            },
            *messages  # Mensajes adicionales del usuario (expansión del mensaje)
        ]
    )

    # Extraer y almacenar los triples (hechos y relaciones) usando el MemoryManager
    memory_manager.invoke({"messages": messages})

    return response  # Devolver la respuesta generada por el modelo



In [7]:
# Invocar la aplicación con un conjunto de mensajes y un contexto configurable que incluye un `user_id`
app.invoke(messages, config={"configurable": {"user_id": "user123"}})


PUT::namespace=('chat', 'user123', 'triples'), key=4107f7c4-87ab-46b9-a487-3700a69d06e7, value={'kind': 'Triple', 'content': {'subject': 'Alice', 'predicate': 'manages', 'object': 'ML team', 'context': None}}:
PUT::namespace=('chat', 'user123', 'triples'), key=e26a6ed7-275d-4a88-8d0a-8a6cb1742efa, value={'kind': 'Triple', 'content': {'subject': 'Alice', 'predicate': 'mentors', 'object': 'Bob', 'context': None}}:
PUT::namespace=('chat', 'user123', 'triples'), key=98ad3d1d-e2a3-4702-ab70-2afbc59d3481, value={'kind': 'Triple', 'content': {'subject': 'Bob', 'predicate': 'is member of', 'object': 'ML team', 'context': None}}:


AIMessage(content="Okay, I understand. Alice is Bob's manager and mentor in the machine learning team.", additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'ca82ef66-e970-4ba6-963e-0b895ae0fd73', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 29 Apr 2025 02:13:16 GMT', 'content-type': 'application/json', 'content-length': '265', 'connection': 'keep-alive', 'x-amzn-requestid': 'ca82ef66-e970-4ba6-963e-0b895ae0fd73'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [1007]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run-cb445bec-4577-4f43-85df-d356d43a0539-0', usage_metadata={'input_tokens': 30, 'output_tokens': 22, 'total_tokens': 52, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})


---

## 🧠 Explicación del código (caso de uso)

### **¿Qué caso de uso cubre este código?**

Este código simula un asistente que **escucha lo que el usuario dice** y **extrae hechos importantes** de esa conversación para guardarlos en memoria.  
**No utiliza** esos hechos guardados aún para responder. **Solo extrae y guarda**, nada más.

**Ejemplo real**:  
Un usuario dice "Alice gestiona el equipo de ML".  
➡ El asistente **responde** normalmente al usuario (con su LLM).  
➡ Y **aparte**, **extrae** el hecho "**Alice - gestiona - equipo de ML**" y lo **guarda** en memoria, por si algún día se quiere usar.

---

### **¿Qué hace el código exactamente?**

1. **Instancia un modelo de lenguaje (LLM)** para que pueda responder preguntas (usa Bedrock y un modelo tipo Claude).
   
2. **Crea un gestor de memoria (Memory Manager)**, que se conecta a una base de memoria (aquí en memoria RAM, `InMemoryStore`).

3. **Define qué se va a extraer**:
   - En este caso, se van a extraer **triples** (sujeto, predicado, objeto).
   - El Memory Manager ya tiene un **prompt especial** que le dice al modelo **qué debe extraer** (factos, relaciones, eventos...).

4. **Cuando llega un mensaje**:
   - **Primero** se manda el mensaje al modelo para que **genere una respuesta** al usuario (**primer `invoke`**).
   - **Luego** se vuelve a mandar el mismo mensaje al Memory Manager, que **usa otra vez el modelo** para **extraer los hechos** y **guardarlos** (**segundo `invoke`**).

---

### **¿Qué **NO** hace este código?**

- **No usa** todavía la memoria para mejorar las respuestas.
- **No busca** en los datos guardados para responder.
- **No hace reasoning** (razonamiento sobre lo que guardó).
- Solo **extrae** y **guarda** datos, como si fueran apuntes rápidos.

---

In [9]:
### Crear un gestor de memoria que no utilice el almacenamiento